In [2]:
import cv2
import numpy as np
import os

In [3]:
def detectSkin(image):
    # Convert the image from BGR color space to YCrCb color space
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    # Define the lower and upper boundaries of the skin color in the YCrCb color space
    lower_skin = np.array([0, 133, 77], dtype=np.uint8)
    upper_skin = np.array([255, 173, 127], dtype=np.uint8)

    # Create a mask of the skin color in the YCrCb color space
    mask = cv2.inRange(ycrcb, lower_skin, upper_skin)

    # Apply the mask to the original image
    skin = cv2.bitwise_and(image, image, mask=mask)

    return skin

In [ ]:
def getTraceFromVidFile(VIDFOLDER, VERBOSE=0):
    # extract RGB traces from vid file
    # contact: yannick.benezeth@u-bourgogne.fr

    # Parameters
    DoSkinDetection = 0  # 0: ROI is the face, 1: ROI is the skin mask
    vidFileName = 'vid.avi'
    outFileName = 'rgbTraces'

    # Create a cascade detector object.
    faceDetector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    # Create the point tracker object.
    pointTracker = cv2.optflow.createTrackerByName('LK')
    
    if (os.path.exists(os.path.join(VIDFOLDER, f'{outFileName}.npy'))):
        print('rgbTraces file exists, just skip getTraceFromVidFile()...')
        return
    
    # Video Handler
    vidObj = cv2.VideoCapture(os.path.join(VIDFOLDER, vidFileName))
    fps = vidObj.get(cv2.CAP_PROP_FPS)
    nbFrame = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
    rgbTraces = np.zeros((4, nbFrame))

    ret, old_frame = vidObj.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    oldPoints = cv2.goodFeaturesToTrack(old_gray, mask=None, maxCorners=100, qualityLevel=0.01,
                                        minDistance=7, blockSize=7, useHarrisDetector=False)

    numPts = 0
    n = 0
    
    while True:
        ret, img = vidObj.read()
        if not ret:
            break
        
        n += 1
        img_copy = img.copy()

        # face localisation by detection then tracking
        if numPts < 10:
            # Detection mode.
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = faceDetector.detectMultiScale(gray, 1.3, 5, minSize=(100, 100))
            
            if len(faces) == 0:
                continue
            
            bbox = faces[0]  # Select first detected face

            scaleX = 1
            scaleY = 1.5
            offsetX = 0.
            offsetY = 0.1
            bbox2 = [max(1, bbox[0] + bbox[2] * (offsetX + (1 - scaleX) / 2)),
                     max(1, bbox[1] + bbox[3] * (offsetY + (1 - scaleY) / 2)),
                     bbox[2] * scaleX, bbox[3] * scaleY]
            bbox = bbox2

            # initialize tracker
            # Find corner points inside the detected region.
            points = cv2.goodFeaturesToTrack(gray, mask=None, maxCorners=100, qualityLevel=0.01, 
                                             minDistance=7, blockSize=7, useHarrisDetector=False)

            # Re-initialize the point tracker.
            xyPoints = points.reshape(-1, 1, 2)
            numPts = xyPoints.shape[0]
            pointTracker.init(img, xyPoints)

            # Save a copy of the points.
            oldPoints = xyPoints.copy()

            # Convert the rectangle represented as [x, y, w, h] into an
            # M-by-2 matrix of [x,y] coordinates of the four corners. This
            # is needed to be able to transform the bounding box to display
            # the orientation of the face.
            x, y, w, h = bbox
            bboxPoints = np.array([[x, y], [x, y + h], [x + w, y + h], [x + w, y]])
            # Crop the face region
            x, y, w, h = bbox
            bboxPoints = np.array([[x, y], [x, y + h], [x + w, y + h], [x + w, y]])
            bboxPoints = np.matmul(np.concatenate((bboxPoints, np.ones((4, 1))), axis=1), M.T)
            bboxPoints = bboxPoints[:, :2]
            bbox = cv2.boundingRect(bboxPoints.astype(int))
            if bbox[0] < 0:
                bbox = (0, bbox[1], bbox[2] + bbox[0], bbox[3])
            if bbox[1] < 0:
                bbox = (bbox[0], 0, bbox[2], bbox[3])
            if bbox[0] + bbox[2] >= img_copy.shape[1]:
                bbox = (bbox[0], bbox[1], img_copy.shape[1] - bbox[0] - 1, bbox[3])
            if bbox[1] + bbox[3] >= img_copy.shape[0]:
                bbox = (bbox[0], bbox[1], bbox[2], img_copy.shape[0] - bbox[1] - 1)
        
            # Crop the face region
            img_copy = img_copy[bbox[1]:bbox[1] + bbox[3], bbox[0]:bbox[0] + bbox[2]]
            
            # If we have a valid face region
            if img_copy.shape[0] != 0 and img_copy.shape[1] != 0:
                # Display the cropped face region
                cv2.imshow('face region', img_copy)
                cv2.waitKey(1)
                # Convert the cropped face region to grayscale
                gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)
                
                # Skin detection
                if DoSkinDetection:
                    # Detect skin color
                    skinMask = detectSkin(gray)
                    # Extract only skin region
                    img_copy = cv2.bitwise_and(img_copy, img_copy, mask=skinMask)
                    # Display skin region
                    cv2.imshow('skin region', img_copy)
                    cv2.waitKey(1)
                
                # Feature detection and tracking
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
                points, status, err = cv2.calcOpticalFlowPyrLK(old_gray, gray, oldPoints, None, **lk_params)
                
                # Update old_frame and oldPoints for the next iteration
                old_gray = gray.copy()
                oldPoints = points.reshape(-1, 1, 2)
                
                # Select good points
                good_new = points[status == 1]
                good_old = oldPoints[status == 1]
                
                # Get the RGB traces for this frame
                R, G, B = getRGBTraces(good_new, good_old, img)
                
                # Store the RGB traces
                rgbTraces[0, n - 1] = R
                rgbTraces[1, n - 1] = G
                rgbTraces[2, n - 1] = B
                rgbTraces[3, n - 1] = n / fps
                
                # Update the previous points
                oldPoints = good_new.reshape(-1, 1, 2)
                numPts = oldPoints.shape[0]
        
        # Display the current frame number
        if VERBOSE and (n % 100 == 0):
            print(f'Frame {n}/{nbFrame} processed...')
        
        # Exit if the user presses the 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video object and close the windows
    vidObj.release()
    cv2.destroyAllWindows()

    # Save the RGB traces to file
    np.save(os.path.join(VIDFOLDER, outFileName), rgbTraces)
    print(f"RGB traces saved to {os.path.join(VIDFOLDER, outFileName)}.npy")

    # Release the video and point tracker objects.
    vidObj.release()
    pointTracker = None
    print('RGB traces extracted.')
